In [1]:
library(lattice)
library(plotrix)
library(ggplot2)

In [2]:
setwd('/panicle/ssapkot/StarchProtein/')

In [74]:
##Functions
#Load pdf2png function for output later
pdf2png <- function( pdf_file, out_size=4000 ){
   out_dir <- "results"
   command <- sprintf('qlmanage -t -s %s -o %s %s', out_size, out_dir, pdf_file)
   system(command)
}

In [59]:
lmm <- read.table('results/GWAS/SAP_GWAS_QK_Starch.assoc.txt', sep='\t', head=T, stringsAsFactors=F)
print('LMM')
head(lmm)
print('BSLMM)')
snp.counts <- read.table('Starch_Parameters_mean.txt', sep='\t', head=T, stringsAsFactors=F)
#colnames(bslmm) <- c('chr','rs','ps', 'alpha', 'beta', 'gamma', 'abs.beta.g')
head(snp.counts)

[1] "LMM"


chr,rs,ps,n_miss,allele1,allele0,af,beta,se,l_remle,l_mle,p_wald,p_lrt,p_score
1,S1_2404,2404,75,C,A,0.056,-0.26540000,0.20330260,3.564568,3.528851,0.1925276,0.1882329,0.1898545
1,S1_2482,2482,10,C,G,0.082,0.18009550,0.20483450,3.577991,3.539679,0.3798324,0.3750854,0.3764754
1,S1_14393,14393,15,T,C,0.225,-0.07239022,0.10565000,3.625595,3.618505,0.4936403,0.4895014,0.4900877
1,S1_14632,14632,93,C,A,0.145,-0.16975850,0.13167990,3.587972,3.553155,0.1981171,0.1937094,0.1951380
1,S1_20388,20388,75,T,A,0.314,0.08032659,0.09727266,3.682073,3.674481,0.4094398,0.4057429,0.4071890
1,S1_23566,23566,60,G,T,0.091,-0.11928630,0.16900880,3.613715,3.576195,0.4807432,0.4761991,0.4769428


[1] "BSLMM)"


CHR,RS,PS,alpha,beta,gamma,abs.beta.g
1,S1_2404,2404,-1.369567e-04,-0.05105861,0.00056810,2.900640e-05
1,S1_2482,2482,1.414969e-04,0.06311729,0.00084780,5.351084e-05
1,S1_14393,14393,9.905805e-05,0.01143702,0.00014065,1.608617e-06
1,S1_14632,14632,-3.015393e-04,-0.06868408,0.00063705,4.375519e-05
1,S1_20388,20388,1.320827e-04,0.01510094,0.00093885,1.417752e-05
1,S1_23566,23566,-1.353825e-04,-0.02690877,0.00043365,1.166899e-05


In [88]:
##LMM PLOT SETUP##
#Calculate genome position (e.g. on Manhattan plot) based on chromosome number and position

chr <- lmm$chr
position <- lmm$ps

get_genome_pos <- function( chr, position, buffer=1e6){
# chr: vector with chromosome numbers
# position: vector with positions, corresponding to each chr_num; adjusted according to max position in snp file
# buffer: space between chromosomes in bp
#     chr_length <- c(
#    chromosome_1 = 73840631,
#    chromosome_2 = 77932606,
#    chromosome_3 = 74441160,  
#    chromosome_4 = 68034345,
#    chromosome_5 = 62352331,
#    chromosome_6 = 62208784,
#    chromosome_7 = 64342021,
#    chromosome_8 = 55460251,
#    chromosome_9 = 59635592,
#    chromosome_10 = 60981646
#  )

chr_length <- c(
   chromosome_1 = 80877469,
   chromosome_2 = 77732218,
   chromosome_3 = 74383873,  
   chromosome_4 = 68651956,
   chromosome_5 = 71847961,
   chromosome_6 = 61272856,
   chromosome_7 = 65459784,
   chromosome_8 = 62670676,
   chromosome_9 = 59410064,
   chromosome_10 = 61226198
 )

    chr_length <- chr_length + buffer

 position + sapply( chr, function(x) sum(c(0,chr_length[-length(chr_length)])[ 1:x ]))
}

lmm$genome_pos <- get_genome_pos(chr, position, buffer=1e6)
nlp <- expression(paste('LMM -log'[10], (italic(p))))
mean_genome_pos <- tapply(lmm$genome_pos, lmm$chr, function(x)(min(x)+max(x))/2)
    
is.odd <- function(x) x %% 2 != 0 
is.even <- function(x) x %% 2 == 0 
lmm$Color <- is.odd(lmm$chr)
lmm$Color <- gsub("TRUE", "orange", lmm$Color)
lmm$Color <- gsub("FALSE", "purple", lmm$Color)
                   

In [61]:
##GEMMA/ Next plot set up ##
#This loop will get the maximum value for each chromosome (the chr. length)
#This loop will get the maximum value for each chromosome (the chr. length)
chr.lengths <- c(0)
for (i in 1:10) {
chr.counts <- subset(snp.counts, snp.counts$CHR==i)
 len <- max(chr.counts$PS)
 addon <- len + chr.lengths[i]
 chr.lengths <- c(chr.lengths, addon)
}
chr.lengths


[1]         0  80877469 158609687 232993560 301645516 373493477 434766333
 [8] 500226117 562896793 622306857 683533055

In [62]:
#Now, you need to add the length of the previous chromosome to the positions of each #SNP in the chromosome that will be plotted next to it
adj.pos <- c()
for (i in 1:nrow(snp.counts)){
pos <- snp.counts[i,3]
curr.chr <- as.numeric(snp.counts[i,1])
new.pos <- pos + chr.lengths[curr.chr]
adj.pos <- c(adj.pos, new.pos)
}
new.snp.counts <- cbind(snp.counts, adj.pos)

g=as.factor(new.snp.counts$CHR)
mean.adj.pos <- tapply( new.snp.counts$adj.pos, new.snp.counts$CHR, function(x) (min(x)+max(x))/2 )

In [63]:
tail(new.snp.counts)
mean.adj.pos

,CHR,RS,PS,alpha,beta,gamma,abs.beta.g,adj.pos
140758,10,S10_61202808,61202808,-8.038018e-05,-0.035040801,0.00058915,2.064429e-05,683509665
140759,10,S10_61211383,61211383,-1.516654e-05,-0.001125336,0.00029355,3.303424e-07,683518240
140760,10,S10_61211391,61211391,-2.403386e-05,-0.006031201,0.00054955,3.314447e-06,683518248
140761,10,S10_61223728,61223728,-1.781894e-04,-0.056391736,0.00050920,2.871467e-05,683530585
140762,10,S10_61226197,61226197,-3.580894e-05,-0.037522038,0.00081040,3.040786e-05,683533054
140763,10,S10_61226198,61226198,3.625113e-04,0.073796878,0.00121000,8.929422e-05,683533055


1         2         3         4         5         6         7         8 
 40439936 119775737 195814508 267324262 337587023 404142448 467504456 531567570 
        9        10 
592610017 652925314

In [89]:
#Making a new column with colors alternating every other chromosome

is.odd <- function(x) x %% 2 != 0
is.even <- function(x) x %% 2 == 0 
new.snp.counts$Color <- is.odd(new.snp.counts$CHR)
new.snp.counts$Color <- gsub("TRUE", "purple", new.snp.counts$Color)
new.snp.counts$Color <- gsub("FALSE", "orange", new.snp.counts$Color)

In [92]:
#Create output file for plot and open a PDF (will close using dev.off() later)
pdf_file <- 'results/Starch_Multi_GWAS.pdf'
pdf(pdf_file, width=13, height=8)

#Create Genome-wise Manhattan Plot
with( lmm, {
   par(mar=c(5,5,4,4)+2)  
   plot( I(-log10(p_score)) ~ I(genome_pos/1e6), cex=0.8,
      col=Color, ylab=nlp, xlab='Chromosome', xaxt='n', main='Starch',
      pch=16, lwd=2, family="Times", las=1, cex.axis=1.5, cex.main=2, cex.lab=1.8, ylim=c(0,10))
   axis(side=1, at=mean_genome_pos/1e6, labels=1:10, tick=F, lwd=2, cex.axis=1.5, family="Times")
   abline(h=-log10(0.05/140e3), col='gray', lty='dashed', lwd=2.5)
	})

#Allows a second plot on the same graph
par(new=TRUE)
#Plot the GEMMA results over the existing LMM plot
plot(new.snp.counts$gamma~new.snp.counts$adj.pos, data=new.snp.counts, col=new.snp.counts$Color, main="", axes=FALSE, xlab="", ylab="", pch=18, type="h", lwd=1.5)
#Put label right side of plot
axis(side=4, ylim=c(0,0.05), las=1, cex.axis=1.3, family="Times")
mtext("BSLMM PIP", side=4, line=4, cex=1.5, family="Times")
abline(h=0.03, col='black', lty='dashed', lwd=1.5)

text(645000000, 0.062, labels="LMM", cex=1.2, family="Times")
text(645000000, 0.06, labels="-  BSLMM", cex=1.2, family="Times")
draw.circle(615000000, 0.062, 0.05, lwd=5)

#Close out pdf output
dev.off()


png 
  2

In [80]:
#Create PNG from PDF to shrink down file size
pdf_file <- 'results/Starch_Multi_GWAS.pdf'
pdf2png(pdf_file , out_size=1000)

Warning message in system(command):
“error in running command”

In [93]:
head(snp.counts[order(-snp.counts$gamma),])

,CHR,RS,PS,alpha,beta,gamma,abs.beta.g
1500,1,S1_4067535,4067535,-0.0010987679,-0.2791097,0.06582045,0.01837112
112819,8,S8_51720767,51720767,-0.0010162805,-0.3052012,0.06408745,0.01955956
115067,8,S8_59121722,59121722,-0.0011287410,-0.2812877,0.06394560,0.01798711
1495,1,S1_4067364,4067364,-0.0010637272,-0.2946239,0.06354205,0.01872101
1496,1,S1_4067377,4067377,-0.0010492161,-0.2828123,0.05326815,0.01506489
112820,8,S8_51721062,51721062,-0.0009855779,-0.3004408,0.05254765,0.01578746
